In [49]:
import json
import pandas as pd
from datetime import datetime, timedelta

#To achieve the desired output, we need to:
#
#1. Load the JSON data.
#
#2. Create an empty DataFrame to store the final result.
#
#3. Iterate over the 'data' list in the JSON data. For each dictionary in the 'data' list:
#   - Extract the 'code' value.
#   - Iterate over the 'history' list. For each dictionary in the 'history' list:
#     - Extract the 'start' and 'interval' values.
#     - Convert the 'start' value to a datetime.
#     - Add the 'interval' value to the 'start' datetime to get the 'end' datetime.
#     - Create a new row in the DataFrame with the 'end' datetime, the 'code' value, and the 'data' value.
#4. Pivot the DataFrame to move the 'code' values into columns.


# Load the data
with open('../data/VIC1.json') as f:
    data = json.load(f)

# Initialize an empty DataFrame
df = pd.DataFrame()

# Iterate over the 'data' list
for d in data['data']:

    start = datetime.fromisoformat(d['history']['start'].replace("Z", "+00:00"))
    # interval is a string like "5m" or "30s" or "1d"
    # add the interval to the start time to get the end time
    interval = int(d['history']['interval'][:-1])
    interval_unit = d['history']['interval'][-1]
    if interval_unit != "m":
        raise "interval unit not supported"
    time = start
    # Iterate over the 'history' data list
    for h in d['history']['data']:
        time = time + timedelta(minutes=int(d['history']['interval'][:-1]))
        row = {'id': d['id'], 'network':d['network'], 'region':d['region'], 'type':d['type'], 'code': d['code'], 'time': time, 'data': h}
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

df.head(6)

,id,network,region,type,code,time,data
0,au.nem.vic1.demand,nem,VIC1,power,demand,2024-01-06 11:05:00+10:00,2995.0
1,au.nem.vic1.demand,nem,VIC1,power,demand,2024-01-06 11:10:00+10:00,3101.0
2,au.nem.vic1.demand,nem,VIC1,power,demand,2024-01-06 11:15:00+10:00,3070.0
3,au.nem.vic1.demand,nem,VIC1,power,demand,2024-01-06 11:20:00+10:00,3169.0
4,au.nem.vic1.demand,nem,VIC1,power,demand,2024-01-06 11:25:00+10:00,3113.0
5,au.nem.vic1.demand,nem,VIC1,power,demand,2024-01-06 11:30:00+10:00,3172.0


In [50]:
# Pivot the DataFrame
df_pivot = df.pivot(index='time', columns='code', values='data')

# Reset the index
df_pivot.reset_index(inplace=True)

df_pivot.head(20)

code,time,086338,AU,battery_charging,battery_discharging,coal_brown,demand,exports,gas_ocgt,hydro,imports,solar_rooftop,solar_utility,wind
0,2024-01-06 11:05:00+10:00,NaN,-23.460,10.830,0.0000,2291.0,2995.0,1209.0,0.0,0.84,-485.3,NaN,694.8,728.6
1,2024-01-06 11:10:00+10:00,NaN,-23.740,1.337,1.5420,2361.0,3101.0,1236.0,0.0,0.83,-453.4,NaN,698.6,743.0
2,2024-01-06 11:15:00+10:00,NaN,0.020,10.120,1.7530,2366.0,3070.0,1192.0,0.0,0.84,-441.6,NaN,657.3,739.2
3,2024-01-06 11:20:00+10:00,NaN,9.000,29.290,0.9084,2384.0,3169.0,1143.0,0.0,0.84,-415.1,NaN,659.6,767.4
4,2024-01-06 11:25:00+10:00,NaN,0.020,56.150,0.0000,2385.0,3113.0,1156.0,0.0,0.84,-488.3,NaN,641.3,770.8
5,2024-01-06 11:30:00+10:00,29.3,1.071,23.970,0.0000,2338.0,3172.0,1163.0,0.0,0.84,-487.8,2801.0,670.0,805.6
6,2024-01-06 11:35:00+10:00,NaN,1.460,3.751,0.0000,2345.0,3182.0,1179.0,0.0,0.84,-501.6,NaN,652.9,807.2
7,2024-01-06 11:40:00+10:00,NaN,9.000,6.638,0.0000,2363.0,3191.0,1197.0,0.0,0.84,-583.7,NaN,607.4,795.8
8,2024-01-06 11:45:00+10:00,NaN,9.000,0.000,5.3240,2470.0,3280.0,1205.0,0.0,0.84,-533.6,NaN,601.7,796.7
9,2024-01-06 11:50:00+10:00,NaN,9.000,29.150,0.0160,2480.0,3266.0,1128.0,0.0,0.84,-495.8,NaN,600.7,791.0
